In [2]:
import featuretools as ft 
data = ft.demo.load_mock_customer()

In [4]:
"""data has three sections customers; sessions; transactions"""

'data has three sections customers; sessions; transactions'

In [7]:
# merging transactions_df

transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])
transactions_df.sample(10) 
#"""Also use : transactions_df.[:10]; for continous data """

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,date_of_birth
143,369,28,2014-01-01 07:02:30,2,148.17,5,mobile,2014-01-01 06:50:35,60091,2010-07-17 05:27:50,1984-07-28
132,261,28,2014-01-01 06:50:35,1,133.71,5,mobile,2014-01-01 06:50:35,60091,2010-07-17 05:27:50,1984-07-28
405,306,29,2014-01-01 07:25:15,4,116.33,1,mobile,2014-01-01 07:10:05,60091,2011-04-17 10:48:33,1994-07-18
10,29,1,2014-01-01 00:10:50,2,20.91,2,desktop,2014-01-01 00:00:00,13244,2012-04-15 23:31:04,1986-08-18
60,397,17,2014-01-01 04:12:25,3,88.97,2,tablet,2014-01-01 04:00:30,13244,2012-04-15 23:31:04,1986-08-18
469,129,34,2014-01-01 08:28:05,3,67.16,3,desktop,2014-01-01 08:24:50,13244,2011-08-13 15:42:34,2003-11-21
38,180,15,2014-01-01 03:48:35,5,146.81,2,desktop,2014-01-01 03:41:00,13244,2012-04-15 23:31:04,1986-08-18
156,149,30,2014-01-01 07:33:55,2,85.35,5,desktop,2014-01-01 07:27:25,60091,2010-07-17 05:27:50,1984-07-28
438,205,19,2014-01-01 04:44:55,3,40.88,3,desktop,2014-01-01 04:27:35,13244,2011-08-13 15:42:34,2003-11-21
299,488,4,2014-01-01 01:03:55,5,129.00,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18


In [8]:
"""Product Dataframe"""
products_df = data["products"]
products_df

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [9]:
"""Creating an EntitySet"""
es = ft.EntitySet(id="customer_data")

In [11]:
'''Adding entities: load the transaction data frame as an entity'''
es = es.entity_from_dataframe(entity_id="transactions",
                             dataframe=transactions_df,
                             index="transaction_id",
                             time_index="transaction_time",
                             variable_types={"product_id": ft.variable_types.Categorical,
                                            "zip_code": ft.variable_types.ZIPCode})
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
  Relationships:
    No relationships

In [15]:
es["transactions"].variables

[<Variable: transaction_id (dtype = index)>,
 <Variable: session_id (dtype = numeric)>,
 <Variable: transaction_time (dtype: datetime_time_index, format: None)>,
 <Variable: amount (dtype = numeric)>,
 <Variable: customer_id (dtype = numeric)>,
 <Variable: device (dtype = categorical)>,
 <Variable: session_start (dtype: datetime, format: None)>,
 <Variable: join_date (dtype: datetime, format: None)>,
 <Variable: date_of_birth (dtype: datetime, format: None)>,
 <Variable: product_id (dtype = categorical)>,
 <Variable: zip_code (dtype = zipcode)>]

In [17]:
#!pip install python-graphviz

es = es.entity_from_dataframe(entity_id="products",
                             dataframe= products_df,
                             index="product_id")
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    No relationships

In [20]:
"""Now We had created two entity sets
Now we can creat a relationship among them"""

new_relationship = ft.Relationship(es["products"]["product_id"],
                                  es["transactions"]["product_id"])
"""Adding Relationship"""
es = es.add_relationship(new_relationship)
es

2019-07-07 00:34:50,615 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: transactions.product_id -> products.product_id>


Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    transactions.product_id -> products.product_id

In [22]:
"""Creating Entity from the existint table"""
"""In order to create the new entity and relationship for sessions we 'normalize'
the transaction entity"""

es = es.normalize_entity(base_entity_id="transactions",
                        new_entity_id="sessions",
                        index="session_id",
                        make_time_index="session_start",
                        additional_variables=["device","customer_id", "zip_code","session_start", "join_date"])

es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 6]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id

In [23]:
es["transactions"].variables

[<Variable: transaction_id (dtype = index)>,
 <Variable: session_id (dtype = id)>,
 <Variable: transaction_time (dtype: datetime_time_index, format: None)>,
 <Variable: amount (dtype = numeric)>,
 <Variable: date_of_birth (dtype: datetime, format: None)>,
 <Variable: product_id (dtype = id)>]

In [24]:
es["sessions"].variables

[<Variable: session_id (dtype = index)>,
 <Variable: device (dtype = categorical)>,
 <Variable: customer_id (dtype = numeric)>,
 <Variable: zip_code (dtype = zipcode)>,
 <Variable: session_start (dtype: datetime_time_index, format: None)>,
 <Variable: join_date (dtype: datetime, format: None)>]

In [25]:
es["products"].variables

[<Variable: product_id (dtype = index)>,
 <Variable: brand (dtype = categorical)>]

In [26]:
"""Looking at the dataframe"""

es["sessions"].df.head(5)

,session_id,device,customer_id,zip_code,session_start,join_date
1,1,desktop,2,13244,2014-01-01 00:00:00,2012-04-15 23:31:04
2,2,mobile,5,60091,2014-01-01 00:17:20,2010-07-17 05:27:50
3,3,mobile,4,60091,2014-01-01 00:28:10,2011-04-08 20:08:14
4,4,mobile,1,60091,2014-01-01 00:44:25,2011-04-17 10:48:33
5,5,mobile,4,60091,2014-01-01 01:11:30,2011-04-08 20:08:14


In [27]:
es["transactions"].df.head(5)

,transaction_id,session_id,transaction_time,amount,date_of_birth,product_id
298,298,1,2014-01-01 00:00:00,127.64,1986-08-18,5
2,2,1,2014-01-01 00:01:05,109.48,1986-08-18,2
308,308,1,2014-01-01 00:02:10,95.06,1986-08-18,3
116,116,1,2014-01-01 00:03:15,78.92,1986-08-18,4
371,371,1,2014-01-01 00:04:20,31.54,1986-08-18,3


In [30]:
"""Last creating customer entity"""
es = es.normalize_entity(base_entity_id="sessions",
                        new_entity_id="customers",
                        index="customer_id",
                        make_time_index="join_date",
                        additional_variables=["zip_code", "join_date"])

es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 4]
    customers [Rows: 5, Columns: 3]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [33]:
"""Now we are ready to our entity set.
Use It with featuretools
Build a feature matrix"""

feature_matrix, feature_defs = ft.dfs(entityset=es, 
                                     target_entity="sessions")

feature_matrix.head(5)

,device,customer_id,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),...,customers.SKEW(transactions.amount),customers.MIN(transactions.amount),customers.MEAN(transactions.amount),customers.COUNT(transactions),customers.NUM_UNIQUE(transactions.product_id),customers.MODE(transactions.product_id),customers.DAY(join_date),customers.YEAR(join_date),customers.MONTH(join_date),customers.WEEKDAY(join_date)
session_id,,,,,,,,,,,,,,,,,,,,,
1,desktop,2,1229.01,41.600976,141.66,0.295458,20.91,76.813125,16,5,...,0.098259,8.73,77.422366,93,5,4,15,2012,4,6
2,mobile,5,746.96,45.893591,135.25,-0.160550,9.32,74.696000,10,5,...,-0.025941,7.55,80.375443,79,5,5,17,2010,7,5
3,mobile,4,1329.00,46.240016,147.73,-0.324012,8.70,88.600000,15,5,...,-0.036348,5.73,80.070459,109,5,2,8,2011,4,4
4,mobile,1,1613.93,40.187205,129.00,0.234349,6.29,64.557200,25,5,...,0.019698,5.81,71.631905,126,5,4,17,2011,4,6
5,mobile,4,777.02,48.918663,139.20,0.336381,7.43,70.638182,11,5,...,-0.036348,5.73,80.070459,109,5,2,8,2011,4,4


In [35]:
feature_matrix, feature_defs = ft.dfs(entityset=es, 
                                     target_entity="products")
feature_matrix.head(5)

,brand,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.session_id),MODE(transactions.session_id),...,MODE(transactions.DAY(transaction_time)),MODE(transactions.DAY(date_of_birth)),MODE(transactions.YEAR(transaction_time)),MODE(transactions.YEAR(date_of_birth)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.MONTH(date_of_birth)),MODE(transactions.WEEKDAY(transaction_time)),MODE(transactions.WEEKDAY(date_of_birth)),MODE(transactions.sessions.device),MODE(transactions.sessions.customer_id)
product_id,,,,,,,,,,,,,,,,,,,,,
1,B,7489.79,42.479989,149.56,0.125525,6.84,73.429314,102,34,3,...,1,18,2014,1994,1,7,2,0,desktop,1
2,B,7021.43,46.336308,149.95,0.151934,5.73,76.319891,92,34,28,...,1,18,2014,2006,1,8,2,0,desktop,4
3,B,7008.12,38.871405,148.31,0.223938,5.89,73.001250,96,35,1,...,1,18,2014,2006,1,8,2,0,desktop,4
4,B,8088.97,42.492501,146.46,-0.132077,5.81,76.311038,106,34,29,...,1,18,2014,1994,1,7,2,0,desktop,1
5,A,7931.55,42.131902,149.02,0.098248,5.91,76.264904,104,34,4,...,1,18,2014,1994,1,7,2,0,mobile,1
